<a href="https://colab.research.google.com/github/Justus-coded/Competitions/blob/master/DSN%20AI%20Bootcamp%20Qualification%20Hackathon/DSN%20AI%20Boot_camp%20Qualification1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Train.csv')
test  = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Test.csv')

In [ ]:
dict_ = {'charge': 1,
         'lending': 0}
 
target_map = {
    'yes': 1,
    'no': 0}
 
train.default_status.replace(target_map, inplace = True)
 
train['form_field47'].replace(dict_, inplace = True)
test['form_field47'].replace(dict_, inplace = True)

In [ ]:
pip install catboost

     |████████████████████████████████| 66.1MB 45kB/s 


In [ ]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [ ]:
features = train.select_dtypes(exclude = object).columns.drop(['default_status'])

In [ ]:
# features

In [ ]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

In [ ]:
X = train[features]
y = train['default_status']

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [ ]:
# Specify number of folds
n_skf = 5
kf = StratifiedKFold(n_skf)
 
 
seed = 2020
 
## Manually tweak the hyperparameter, learning rate and n_estimators especially
params = {
    'n_estimators': 4000,
    'learning_rate': 0.01,
    'objective': 'CrossEntropy',
      'eval_metric':'AUC',
    'random_seed': seed,
    'early_stopping_rounds': 200,
    'use_best_model': True,
}

In [ ]:
score_list = []
score = 0
test_oofs = []
 
for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]
 
    model = CatBoostClassifier(**params, task_type='GPU')
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p = model.predict_proba(xval)[:, 1]
    sc = metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)
 
    print('Fold {} : {}'.format(i, sc))
 
print()
print()
print('Avg log : ', score)

0:	learn: 0.7882729	test: 0.7847513	best: 0.7847513 (0)	total: 99.2ms	remaining: 6m 36s
100:	learn: 0.8290880	test: 0.8248354	best: 0.8248354 (100)	total: 5.3s	remaining: 3m 24s
200:	learn: 0.8348939	test: 0.8293202	best: 0.8293202 (200)	total: 10.4s	remaining: 3m 15s
300:	learn: 0.8380512	test: 0.8316320	best: 0.8316320 (300)	total: 15.3s	remaining: 3m 8s
400:	learn: 0.8403282	test: 0.8329767	best: 0.8329767 (400)	total: 20.2s	remaining: 3m 1s
500:	learn: 0.8421736	test: 0.8339559	best: 0.8339559 (500)	total: 25.3s	remaining: 2m 56s
600:	learn: 0.8436579	test: 0.8346310	best: 0.8346310 (600)	total: 30.2s	remaining: 2m 50s
700:	learn: 0.8449304	test: 0.8351660	best: 0.8351660 (700)	total: 35.1s	remaining: 2m 45s
800:	learn: 0.8461992	test: 0.8356704	best: 0.8356712 (795)	total: 40s	remaining: 2m 39s
900:	learn: 0.8471835	test: 0.8359593	best: 0.8359604 (898)	total: 44.9s	remaining: 2m 34s
1000:	learn: 0.8481769	test: 0.8362668	best: 0.8362668 (1000)	total: 49.9s	remaining: 2m 29s
1100:

In [ ]:
f"{n_skf} fold CV, score: {score}" #Increase in local cv score should probably lead to increase in Leaderboard score

'5 fold CV, score: 0.8403491472009266'

### Since we are using cross validation, We'll have `n` different predictions (where n is the number of fold)

In [ ]:
oof_prediction = pd.DataFrame(test_oofs).T

In [ ]:
oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [ ]:
oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5
0,0.333799,0.336533,0.368278,0.349320,0.340148
1,0.453731,0.379594,0.378314,0.321132,0.360573
2,0.338883,0.390586,0.429316,0.399253,0.345758
3,0.724276,0.715292,0.717721,0.745203,0.729277
4,0.215902,0.197030,0.166876,0.141364,0.166670


In [ ]:
sub = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/SampleSubmission.csv')

In [ ]:
### We take the mean of all 5 predictions to generate submission

In [ ]:
sub['default_status'] = np.mean(test_oofs, axis = 0)

In [ ]:
sub.to_csv('submit1.csv', index = False)

### Suggestions:
- Base your feature selection on the local cv score i.e When you add, remove or engineer feature check `score` (cell 17)
- Increase number of fold to 10, 15. See how score imporve on leaderboard.
- Try differnt value of learning rate. I'll advice anything < 0.01
- Create as many features as possible.
- Find code for Xgboost and lgbm using this method of modelling, or better still tweak this one (mind you, names of parameters may be slightly different.